In [1]:
!pip install -U \
  numpy \
  librosa \
  soundfile \
  torch \
  torchaudio \
  transformers \
  scikit-learn \
  faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 853.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 151.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 66.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 13.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 58.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U torch==2.9.0 torchaudio==2.9.0 torchvision==0.24.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 27.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.6 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: triton
    Found existing installation: triton 3.5.1
    Uninstalling triton-3.5.1:
      Successfully uninstalled triton-3.5.1
  Attempting uninstall: torch
    Found existing installation: torch 2.9.1
    Uninstalling torch-2.9.1:
      Successfully uninstalled torch-2.9.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.9.1
    Uninstalling torchaudio-2.9.1:
      Successfully uninstalled torchaudio-2.9.1


# Global Setup

Cell 1 initializes the AudioTrust environment by enforcing reproducibility, deterministic GPU behavior, and Colab-safe defaults, while defining global audio and training assumptions such as sample rate, maximum audio duration, and batch size. This cell ensures that all experiments are stable, repeatable, and consistent across runs, and it deliberately avoids any data loading, feature extraction, or model logic so that these global settings never need to change as the project evolves.

In [3]:
# =========================================================
# AudioTrust — Global Setup (Colab-First)
# =========================================================

import os
import random
import numpy as np
import torch

# -------------------------
# Reproducibility
# -------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# -------------------------
# Device configuration
# -------------------------
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

# -------------------------
# Audio configuration
# -------------------------
SAMPLE_RATE = 16000          # Required for ASVspoof & SSL models
MAX_DURATION = 6.0           # seconds (explicit, NOT implicit)
MAX_SAMPLES = int(SAMPLE_RATE * MAX_DURATION)

# -------------------------
# Feature configuration
# -------------------------
N_FFT = 1024
HOP_LENGTH = 256
N_MELS = 80

# -------------------------
# Training configuration
# -------------------------
BATCH_SIZE = 16              # Colab-safe
NUM_WORKERS = 2              # Avoid Colab crashes

print("Global config loaded.")

Using device: cuda
Global config loaded.


In [4]:
import torchaudio
from torch.utils.data import Dataset


# ASVspoof Dataset

Cell 2 defines how raw ASVspoof audio enters the AudioTrust system by implementing a PyTorch Dataset that parses official protocol files for labels, loads audio as waveforms, standardizes it to mono 16 kHz audio, and applies explicit padding or truncation to a fixed duration. This waveform-first, framework-native design avoids hardcoded spectrogram assumptions, supports real-time inference later, and provides a clean, reusable data–model interface for CNNs, VAEs, GANs, and self-supervised speech models.

In [5]:
import os
import torch
import torchaudio
from torch.utils.data import Dataset


class ASVspoofDataset(Dataset):
    """
    PyTorch Dataset for ASVspoof (2019 / 2021).

    Returns:
        waveform: Tensor of shape (max_samples,)
        label: 0 = bonafide (real), 1 = spoof (fake)
    """

    def __init__(
        self,
        audio_dir: str,
        protocol_file: str,
        sample_rate: int,
        max_samples: int,
    ):
        self.audio_dir = audio_dir
        self.sample_rate = sample_rate
        self.max_samples = max_samples

        # -------------------------------
        # Parse ASVspoof protocol file
        # -------------------------------
        self.file_labels = {}

        with open(protocol_file, "r") as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) < 5:
                    continue

                file_id = parts[1]            # e.g. LA_T_1000137
                label_str = parts[-1]         # bonafide / spoof
                label = 0 if label_str == "bonafide" else 1

                self.file_labels[file_id] = label

        self.files = sorted(self.file_labels.keys())

        print(f"[ASVspoofDataset] Loaded {len(self.files)} samples")

    def __len__(self):
        return len(self.files)

    def _load_audio(self, file_path: str) -> torch.Tensor:
        """
        Load audio, convert to mono, resample, pad/truncate.
        """
        waveform, sr = torchaudio.load(file_path)  # (C, T)

        # Convert to mono
        if waveform.shape[0] > 1:
            waveform = waveform.mean(dim=0, keepdim=True)

        # Resample if needed
        if sr != self.sample_rate:
            waveform = torchaudio.functional.resample(
                waveform, sr, self.sample_rate
            )

        waveform = waveform.squeeze(0)  # (T,)

        # Pad or truncate to fixed length
        if waveform.shape[0] > self.max_samples:
            waveform = waveform[: self.max_samples]
        else:
            pad_len = self.max_samples - waveform.shape[0]
            waveform = torch.nn.functional.pad(waveform, (0, pad_len))

        return waveform

    def __getitem__(self, idx):
        file_id = self.files[idx]
        label = self.file_labels[file_id]

        audio_path = os.path.join(self.audio_dir, file_id + ".flac")
        waveform = self._load_audio(audio_path)

        return waveform, label
